# Fase 2: Creazione del backend del Chatbot (da svolgere entro Lunedì 20 Ottobre)

Prima di iniziare il notebook consultare le slides a questo link: https://docs.google.com/presentation/d/1OFu-hU7-CMQR6hB7ydX2pOxRf5KWLdLV2P9ePetcW0k/edit?usp=sharing

Le slides sono state spiegate nella lezione "Spiegazione dei laboratori sulla creazione del ChatBot", presente sul portale della didattica.

Il codice seguente mostra un esempio di creazione di un chatbot a partire da un file PDF. Partite da questo notebook per creare il vostro con i vostri documenti.

**INDICAZIONI PER LA CONSEGNA**:

1.   **IMPORARTE TUTTI I VOSTRI FILE, NON SOLO UN PDF COME IN QUESTO ESEMPIO;**
2.   **ALLA FINE DEL NOTEBOOK SALVARE LE RISPOSTE DEL CHATBOT A TUTTE LE 10 DOMANDE DEFINITE NELLA FASE PRECEDENTE DEL PROGETTO IN UN FILE CSV; IL CSV DEVE ESSERE FORMATO USANDO LA LIBRERIA PANDAS E DEVE AVERE DUE COLONNE NOMINATE "domanda E "risposta";**
3. **ALLA FINE DEL LAVORO: A) CONTROLLARE CHE TUTTO IL NOTEBOOK FUNZIONI CORRETTAMENTE RILANCIANDOLO E VERIFICANDO CHE PRODUCA COME OUTPUT IL CSV CON LE RISPOSTE; B) SCARICARE SIA IL NOTEBOOK CHE IL CSV; 3) SALVARLI IN UNA CARTELLA COL NOME DEL VOSTRO CHATBOT COME NEL LABORATORIO PRECEDENTE; 4) CARICARLA NELLA SEZIONE ELABORATI, NON ELIMINANDO QUELLA PRECEDENTE ANCHE SE HANNO LO STESSO NOME.**

## Importazione dei pacchetti Python utili

In [1]:
!pip install pypdf
!pip install langchain
!pip install nltk
!pip install huggingface_hub
!pip install sentence_transformers
!pip install chromadb==0.4.15
!pip install tiktoken
!pip install peft
!pip install googletrans==3.1.0a0
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=043db10fca805a9a8cd574e2185b92f66e6f6a9001d18fe318efb1ae3a2b70de
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.2 MB/s e

## Caricamento dei documenti
La fase del caricamento dei documenti è già stata vista nel notebook precedente, precisamente nella sezione **Definire i documenti**. In questo esempio di codice importerò soltanto un PDF da Github; ricordatevi che **i documenti dovranno essere minimo 3 e massimo 6, tra cui almeno un PDF ed una pagina web. SOSTITUIRE IL CODICE SEGUENTE CON IL VOSTRO PER CARICARE TUTTI I DOCUMENTI!**
Link utili:
*   Caricamento PDF: https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf
*   Caricamento HTML: https://python.langchain.com/docs/modules/data_connection/document_loaders/html

In [2]:
!git clone https://github.com/MelissaJolanda/files_for_chatbot.git

Cloning into 'files_for_chatbot'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 50 (delta 6), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (50/50), 11.13 MiB | 15.57 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [3]:
from langchain.document_loaders import UnstructuredURLLoader

# Caricamento files HTML

# Definisci una lista di URL da cui scaricare contenuto
urls = [
    "https://www.pianetadesign.it/catalogo/ikea-ufficio-la-nuova-collezione-trotten.php",
    "https://www.ikea.com/it/it/rooms/home-office/how-to/come-realizzare-la-scrivania-da-gaming-perfetta-per-te-pub147a6c80",
    "https://www.ikea.com/it/it/cat/uppspel-serie-54984/"
]
# Crea un oggetto 'loader' di tipo UnstructuredURLLoader, specificando gli URL da caricare
loader1 = UnstructuredURLLoader(urls=urls)
# Carica il contenuto dalle URL specificate
documents1 = loader1.load()

#Ho settato solo 3 documenti da visualizzare siccome il file caricato è un PDF con almeno 200 pagine e ogni pagina viene salvata come un documento
# Per visualizzare solo i primi 3 documenti (pagine) dal PDF
for doc in documents1[:4]:
    # Estrai il contenuto della pagina
    page_content1 = doc.page_content

    # Ottieni la fonte del documento dalla metadati
    source1 = doc.metadata["source"]

    # Estrai le prime 100 parole dal contenuto della pagina
    first_words1 = " ".join(page_content1.split())

    # Stampa le prime 100 parole della pagina insieme alla sua fonte
    print(f"{source1}:\n{first_words1}")
    print("\n\n")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


https://www.pianetadesign.it/catalogo/ikea-ufficio-la-nuova-collezione-trotten.php:
Design Interni Hospitality Storytelling Designer Arredo Lifestyle Visioni Arredamento Zona giorno Zona notte Cucina Bagno Outdoor Illuminazione Elementi di arredo Domotica News Fiere ed Eventi Aziende Normative Cataloghi Fai da te Video Seguici su Chi siamo Redazione Contatti © 2023 - P.IVA 04827280654 - Testata Registrata al tribunale di Nocera Inferiore n. 8/2020 - RG n. 1336/2020 - ISCRIZIONE AL ROC N. 35792 – ISCRITTA ALL’ANSO (ASSOCIAZIONE NAZIONALE STAMPA ONLINE) Privacy e Notifiche Preferenze privacy Mappa del sito SCOPRI "VISIONI" LA NUOVA RUBRICA DI PIANETADESIGN IKEA ufficio Home » Cataloghi » Ikea ufficio: la nuova collezione Trotten Ikea ufficio: la nuova collezione Trotten Autore: Cantoni Stefano Autore Segnala modifica Pronta la collezione nuova per ufficio Trotten firmata Ikea: scopriamo tutto su questi complementi di arredo versatili e unici. Trotten consente di lavorare sia in ufficio c

In [4]:
#Importazione delle librerie PyPDFLoader e os
from langchain.document_loaders import PyPDFLoader
import os

# Definisci il percorso alla cartella contenente i file PDF da elaborare
path_to_PDF_files = "files_for_chatbot/PDF/"
# Ottieni la lista di tutti i file nella cartella specificata
files = os.listdir(path_to_PDF_files)

# Crea una lista vuota chiamata 'documents' per contenere i documenti estratti dai file PDF
documents = list()

# Loop attraverso ogni file nella lista 'files'
for f in files:
    # Crea un oggetto 'loader' per PyPDFLoader specifico per il file PDF corrente
    loader = PyPDFLoader(path_to_PDF_files + f)
    # Carica e suddivide il contenuto del file PDF corrente
    documents += loader.load_and_split()
    print(f)

#Ho settato solo 3 documenti da visualizzare siccome il file caricato è un PDF con almeno 200 pagine e ogni pagina viene salvata come un documento
# Per visualizzare solo i primi 3 documenti (pagine) dal PDF
for doc in documents[:2]:
    # Estrai il contenuto della pagina
    page_content = doc.page_content

    # Ottieni la fonte del documento dalla metadati
    source = doc.metadata["source"]

    # Estrai le prime 100 parole dal contenuto della pagina
    first_words = " ".join(page_content.split())

    # Stampa le prime 100 parole della pagina insieme alla sua fonte
    print(f"{source}:\n{first_words}")
    print("\n\n")



scrivanieBambu2.pdf
scrivaniaPerfettagiusta.pdf
files_for_chatbot/PDF/scrivanieBambu2.pdf:
scrivanie in bambù: NORDKISA Toeletta, 76x47 cm €119. Disponibile per la consegna. Non in v endita pr esso T orino Collegno. ANFALLAREPiano tavolo, bambù, 140x65 cm €80 . Disponibile per la consegna. Disponibile pr esso T orino Collegno LILLÅSEN Scrivania, bambù, 102x49 cm €149 . Disponibile per la consegna. Disponibile pr esso T orino Collegno. ANFALLARE / HILVER Scrivania, bambù, 140x65 cm €140. Attualmente non disponibile per la consegna.Esaurit o presso T orino Collegno. ANFALLARE / ALEX Scrivania, bambù, 140x65 cm €170. Disponibile per la consegna. Esaurit o presso T orino Collegno. ANFALLARE / ADILS Scrivania, bambù, 140x65 cm €100. Disponibile per la consegna. Disponibile pr esso T orino Collegno ANFALLARE / MITTBACK Scrivania, bambù, 140x65 cm €160 . Disponibile per la consegna. Disponibile pr esso Torino Collegno ANFALLARE / OLOV Scrivania, bambù, 140x65 cm €140. Disponibile per la conse

In [5]:
#Unione di files HTML e PDF in una sola lista di documenti

for doc in documents1:
  documents.append(doc)

## Suddivisione dei documenti in chunks


Lo "splitter del testo" (text splitter) è importante per due motivi principali:

* **Retrieval Augmented Generation**: nelle applicazioni in cui si utilizza una combinazione di generazione di testo e recupero di contenuti, il text splitter è essenziale per recuperare solo i segmenti di contenuto più rilevanti. Ad esempio, in un sistema di generazione di testo assistito dal recupero, potresti voler recuperare informazioni pertinenti da una vasta collezione di documenti o testi prima di generare un nuovo testo. Lo splitter consente di suddividere il testo da recuperare in parti più piccole e quindi eseguire il recupero solo di quelle parti che sono rilevanti per il contesto o l'obiettivo specifico. Questo rende il processo di generazione finale più efficace siccome non considera le parti meno semnaticamente legate alla domanda dell'utente.

* **Lunghezza massima dei token**: i testi verranno salvati all'interno del Vectorstore come vettori di numeri. Per ottenere questa rappresentazione è necessario utilizzare un Sentence Trasformer, che ha una lunghezza massima consentita per le sequenze di token che possono essere elaborate. Questo significa che se un testo è troppo lungo e supera questa lunghezza massima, sarà necessario "tagliare" o "troncare" il testo per farlo rientrare nella dimensione massima consentita. Ciò può comportare la perdita di informazioni importanti. Il text splitter consente di dividere il testo in segmenti più brevi, rispettando la lunghezza massima dei token. In questo modo, il modello può rappresentare ciascun segmento separatamente senza tagliare parti importanti del testo originale.



Per splittare i documenti in chunks è possibile usare diversi metodi, consultare: https://python.langchain.com/docs/modules/data_connection/document_transformers/

Nel mio esempio ho creato un metodo customizzato. La funzione `split_text_into_groups` accetta un testo e lo suddivide in gruppi di frasi. Puoi specificare il numero di frasi da includere in ciascun gruppo utilizzando il parametro `n`, mentre il parametro `overlap` controlla quante frasi di sovrapposizione ci sono tra gruppi consecutivi. Di default, è impostato su 2. La funzione utilizza il tokenizzatore di frasi nltk per suddividere il testo in frasi, e quindi crea gruppi unendo un certo numero di frasi in base ai parametri specificati. Questa suddivisione del testo in gruppi può essere utile per analizzare testi lunghi o per lavorare con parti specifiche di un testo in modo più gestibile. La funzione restituisce una lista di gruppi di frasi come risultato.

In [6]:
import nltk
nltk.download('punkt')

def split_text_into_groups(text, n=4, overlap=1, lang="italian"):
    """
    Questa funzione suddivide un testo in gruppi di frasi.

    Argomenti:
    - text (str): Il testo da suddividere in gruppi.
    - n (int): Il numero di frasi da includere in ciascun gruppo.
    - overlap (int): Il numero di frasi di sovrapposizione tra gruppi consecutivi.
    - lang (str): La lingua del testo (impostata su "italian" di default).

    Restituisce:
    - groups (list): Una lista di gruppi di frasi.

    Esempio di utilizzo:
    text = "Questo è un esempio di testo da suddividere in gruppi. Può essere utile per l'analisi testuale."
    groups = split_text_into_groups(text, n=3, overlap=2)
    """
    # Tokenizza il testo in frasi sulla base della lingua specificata
    sentences = nltk.sent_tokenize(text, language=lang)

    # Inizializza una lista per i gruppi di frasi
    groups = []

    # Suddivide il testo in gruppi
    for i in range(0, len(sentences), n - overlap):
      if i + n < len(sentences):
        group = ' '.join(sentences[i:i + n])
        groups.append(group)

    # Restituisce i gruppi di frasi
    return groups



from langchain.docstore.document import Document

# Lista vuota per memorizzare i documenti divisi
documents_splitted = list()

# Contatore per generare un ID univoco per ciascun documento
document_id = 0

# Itera sui primi 20 documenti per velocizzare l'elaborazione
for original_doc in documents:
    # Split del contenuto del documento in gruppi
    for group_content in split_text_into_groups(original_doc.page_content):

        # Copia dei metadati originali del documento
        metadata = original_doc.metadata

        # Assegna un ID univoco al documento
        metadata['id'] = document_id

        # Incremento dell'ID per il prossimo documento
        document_id += 1

        # Creazione di un nuovo documento con il contenuto diviso e i metadati aggiornati
        doc_splitted = Document(page_content=group_content, metadata=metadata)

        # Aggiungi il documento diviso alla lista
        documents_splitted.append(doc_splitted)
print(len(documents_splitted))

36


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Nel codice seguente, traduco il contenuto dei chunks dall'italiano all'inglese. Questa scelta è motivata dal fatto che molti modelli di linguaggio, tra cui i modelli di Sentence Transformer e i large language model, sono stati inizialmente allenati principalmente in lingua inglese. Di conseguenza, questi modelli tendono ad avere performance migliori nella comprensione, generazione e elaborazione di testo in inglese rispetto ad altre lingue.

In [7]:
from googletrans import Translator

# Definizione di una funzione che utilizza googletrans per tradurre il testo
def google_translator(text, from_code="it", to_code="en"):
    translator = Translator()
    # Traduzione del testo dalla lingua di origine alla lingua di destinazione
    translation = translator.translate(text, dest=to_code, src=from_code).text
    return translation

# Lista vuota per memorizzare i documenti tradotti in inglese
documents_splitted_en = list()

# Iterazione su una lista di documenti suddivisi
for doc in documents_splitted:
    # Traduzione del contenuto della pagina del documento corrente dall'italiano all'inglese
    doc_splitted_en = Document(
        page_content=google_translator(doc.page_content),
        metadata=doc.metadata
    )
    # Aggiunta del documento tradotto (in inglese) alla lista
    documents_splitted_en.append(doc_splitted_en)



## Definizione del modello per la creazione degli embeddings


Una volta ottenuti i gruppi/chunks di testo finali è necessario verificare che nessuno di essi ecceda la lunghezza massima del Sentence Transformer in numero di token.

Quindi, nel codice seguente, la funzione `get_model_max_len` restituisce la lunghezza massima consentita dei token per il modello Sentence Transformer  specificato nella variabile `embeddings_model_name`. Conoscere questa lunghezza massima aiuta a determinare quanto lungo può essere il testo senza venire troncato.

Invece la funzione `count_text_token` conta il numero di token in un testo specifico fornito come parametro text, utilizzando il tokenizer associato al modello Sentence Transformer specificato tramite il parametro model_name. Questo è importante perché aiuta a comprendere la lunghezza effettiva del testo in termini di token. Questa informazione è cruciale per determinare se il testo supera la lunghezza massima dei token consentita dal modello e se è necessario segmentarlo o adottare altre strategie per gestire testi troppo lunghi.

Nell'esempio sottostante è stato utilizzato il modello di Sentence Transformer multilingua "all-mpnet-base-v2", tuttavia potete sceglierne altri al seguente link: https://huggingface.co/models?library=sentence-transformers ; modificando i parametri di ricerca potete anche selezionare soltanto i modelli che operano per una determinata lingua.

In [8]:
embeddings_model_name = "all-mpnet-base-v2"#"paraphrase-multilingual-mpnet-base-v2"

In [27]:
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer, util

# Funzione per ottenere la lunghezza massima consentita dal modello di embedding
def get_model_max_len(model_name=embeddings_model_name):
    # Carica il modello SentenceTransformer specificato
    model = SentenceTransformer('sentence-transformers/' + model_name)

    # Accedi al tokenizer dal modello SentenceTransformer
    tokenizer = model.tokenizer

    # Ottieni la lunghezza massima consentita dal modello
    max_len = tokenizer.model_max_length

    return max_len

# Funzione per contare il numero di token in un testo utilizzando il tokenizer del modello
def count_text_token(text, model_name=embeddings_model_name):
    # Carica il tokenizer del modello specificato
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/' + model_name)

    # Tokenizza il testo utilizzando il tokenizer del modello
    tokens = tokenizer(text)

    # Conta il numero di token nel testo tokenizzato
    num_tokens = len(tokens['input_ids'])

    return num_tokens

# Ottieni la lunghezza massima consentita dal modello di embedding
max_len_model = get_model_max_len()

# Itera su ciascun documento nella lista "documents_splitted"
for doc in documents_splitted:
    text = doc.page_content

    # Contatore per generare un ID univoco per ciascun documento
    document_id = len(documents_splitted)
    # Verifica se il numero di token nel testo supera la lunghezza massima del modello
    if count_text_token(text) > max_len_model:
        #suddivide ulteriormente il documento troppo lungo
        for group_content in split_text_into_groups(text):
          # Copia dei metadati originali del documento
          metadata = doc.metadata

          # Assegna un ID univoco al documento
          metadata['id'] = document_id

          # Incremento dell'ID per il prossimo documento
          document_id += 1

          # Creazione di un nuovo documento con il contenuto diviso e i metadati aggiornati
          doc_splitted = Document(page_content=group_content, metadata=metadata)

          # Aggiungi il documento diviso alla lista
          documents_splitted.append(doc_splitted)

        #toglie il documento che eccedeva in lunghezza dalla lista
        documents_splitted.remove(doc)



for doc in documents_splitted:
  text=doc.page_content
  if count_text_token(text) > max_len_model:
    print(f" {doc.metadata}: Il testo supera la lunghezza massima del modello di embedding")



Viene infine definto il caricato il modello per gli embeddings che userà la langchain.

In [28]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name=embeddings_model_name,
    model_kwargs={"device": "cpu"},
    encode_kwargs={'normalize_embeddings': True}
)

## Creazione del Vectorstore

Una delle modalità più comuni per archiviare e cercare dati non strutturati consiste nel convertirli in vettori di embedding e archiviare i vettori risultanti. Successivamente, durante la fase di interrogazione, si effettua l'embedding della query non strutturata e si recuperano i vettori di embedding che sono 'più simili' alla query incorporata. Un sistema di archiviazione vettoriale (vectorstore) si occupa di memorizzare i dati incorporati e di eseguire la ricerca vettoriale per te.


In questa guida, viene utilizzato il database vettoriale Chroma.

In [29]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(documents_splitted_en, embedding_model,collection_metadata={"hnsw:space": "cosine"})


## Definizione del large language model

In questa fase, viene definito il large language model che la chat utilizzerà per il recupero dei file. Nell'esempio fornito viene utilizzato il modello "declare-lab/flan-alpaca-large". Tuttavia potete selezionarne altri a questo link: https://huggingface.co/models?pipeline_tag=text2text-generation&sort=downloads

In [30]:
# Creazione di una directory per memorizzare i file del modello
!mkdir flan-alpaca-large

# Importazione della funzione snapshot_download dal modulo huggingface_hub
from huggingface_hub import snapshot_download

# Scaricamento del modello da Hugging Face Model Hub e memorizzazione nella directory locale
snapshot_download(repo_id="declare-lab/flan-alpaca-large", local_dir='./flan-alpaca-large/', local_dir_use_symlinks=False, token="hf_...")

# Importazione della classe HuggingFacePipeline dal modulo langchain.llms
from langchain.llms import HuggingFacePipeline

# Specifica del percorso alla directory contenente i file del modello
model_dir = "./flan-alpaca-large/"

# Inizializzazione di una pipeline per l'uso del modello scaricato
llm = HuggingFacePipeline.from_model_id(model_id=model_dir,
                                        task='text2text-generation',
                                        model_kwargs={ "min_length": 35, "max_length": 500, "repetition_penalty": 5.0})


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.84k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

# Definizione del retriever

Questo codice crea un oggetto qa di tipo RetrievalQA configurato con le opzioni e le risorse specificate. La classe RetrievalQA è utilizzata per eseguire operazioni di domanda e risposta basate su modelli di linguaggio, utilizzando un retriever per recuperare documenti pertinenti.

In [31]:
# Importazione delle librerie e classi necessarie
from langchain.chains import RetrievalQA  # Importa la classe RetrievalQA dal modulo langchain.chains
from langchain.llms import OpenAI  # Importa la classe OpenAI dal modulo langchain.llms

# Creazione di un oggetto RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,  # Specifica il modello di linguaggio da utilizzare, precedentemente inizializzato
    chain_type="stuff",  # Tipo di catena (chain) da utilizzare (può variare in base all'applicazione)
    retriever=db.as_retriever(),  # Configura il chromadb precedentemente creato come retriever
    return_source_documents=True,  # Opzione per restituire i documenti sorgente
    verbose=False  # Disattiva la modalità verbosa (output dettagliato)
)

# Test del chatbot

Nel codice seguente viene testato il chatbot con una query, utilizzando la funzione `get_chat_response`. **VI RICORDO CHE VOI DOVRETE FARLO PER TUTTE LE DOMANDE DEFINITE NEL VOSTRO CSV, SALVDANDOVI LE RISPOSTE IN UN DATAFRAME PANDAS FORMATO DA DUE COLONNE CHIAMATE "domanda" E "risposta"**. AIUTO: https://chat.openai.com/share/79ea3308-e98f-4cee-aa19-bbd01906c6ee


In [34]:
import warnings
import pandas as pd

# Funzione per ottenere una risposta dalla chat
def get_chat_response(text, qa=qa, lang='it'):
    # Disabilita i warning all'interno della funzione
    with warnings.catch_warnings():
      warnings.filterwarnings("ignore")
      if lang == "it":
          # Traduci il testo in inglese utilizzando google_translator
          text_en = google_translator(text)

          # Esegui una query usando la pipeline di RetrievalQA (qa) sul testo in inglese
          res = qa(
              text_en
          )

          # Traduci la risposta in italiano
          res["query"] = text
          res["result"] = google_translator(res["result"], from_code="en", to_code="it")
      else:
          # Esegui una query usando la pipeline di RetrievalQA (qa) sul testo in lingua originale
          res = qa(
              text
          )
    return res

# Creo una lista con le query che mi servono
lista_domande = ['Quali sono gli arredamenti essenziali per un ufficio?', 'Quali sono le scrivanie in bamboo?', 'Qual è la sedia da gaming più economica?', 'Quali sono le sedie da gaming disponibili a Collegno?', 'Qual è la sedia da gaming più costosa?', 'Cosa dovrebbe avere la mia postazione da gaming ?', 'Qual è la cassettiera adatta alla mia postazione da gaming?', 'Qual è la miglior scrivania da gaming?', 'Quali sono gli articoli da gaming?', 'Qual è la vetrinetta adatta ad un ambiente da gaming?']
lista_risposte = []

# Creo un DataFrame Pandas, usando un dizionario
data_dict = {'Domanda': [], 'Risposta': []}
df1 = pd.DataFrame(data_dict)


for query in lista_domande:

  print("\nQuery:", query)

  # Ottenimento della risposta utilizzando la funzione get_chat_response
  res = get_chat_response(query)

  # Estrazione del testo di risposta
  response_text = res['result']

  # Aggiungo la risposta alla lista
  lista_risposte.append(response_text)

  print("Risposta:", response_text)


#Stampo la tabella con tutte le domande e le relative risposte
df1['Domanda'] = lista_domande
df1['Risposta'] = lista_risposte
#print(f'\n{df1}')



Query: Quali sono gli arredamenti essenziali per un ufficio?
Risposta: Catalogo IKEA Trotten: mobili per ufficio. Le lavagne con set portaoggetti sono essenziali e versatili, oltre che molto apprezzate dai clienti che apprezzano l'utilità nella progettazione estetica per un gioco piuttosto che per un vero e proprio lavoro o ambiente di lavoro.

Query: Quali sono le scrivanie in bamboo?
Risposta: Le scrivanie in bambù sono mobili per ufficio modulari e versatili che possono essere utilizzati per vari scopi. Sono disponibili in varie dimensioni, forme, colori e finiture per adattarsi alle esigenze o allo stile del tuo spazio di lavoro.

Query: Qual è la sedia da gaming più economica?
Risposta: Non lo so. La sedia da gaming più economica è quella con altezza e braccioli regolabili, ma potrebbe non essere altrettanto comoda o ergonomica quando si gioca per lunghi periodi di tempo a causa delle sue caratteristiche limitate, come l'assenza di supporto per la schiena (che potrebbe causare di